In [2]:
%%capture
!pip install transformers

In [3]:
import transformers

In [4]:
art1='The 11th round of India-China military dialogue may take place on April 9 to resolve the friction at Gogra-Hot Springs and restoring the pre-April 2020 status quo ante along the 1,597-kilometre Line of Actual Control (LAC) in East Ladakh, people familiar with the matter said on Tuesday. The Indian Army also wants to resolve long-standing patrolling issues at Depsang Bulge due to friction in the area during the 2013 stand-off with the People’s Liberation Army. “The dialogue will most probably take place on Friday (April 9) but a confirmation is still awaited from the PLA side,” said a senior official. He, however, insisted that there is no delay from the Chinese side. While the situation on both banks of Pangong Tso is stable with the PLA and Indian Army going back to pre-April 2020 positions, a score of Chinese soldiers is still to restore status quo ante in the Gogra-Hot Springs area, or patrolling points 15 and 17 as per the Indian military lexicon. The last round of military talks to resolve border tensions that began last May was held on February 20. This meeting stretched for 16 hours and ended with two sides agreeing that the Pangong Tso disengagement provided them with a “good basis” to resolve the outstanding issues at friction points in “a steady and orderly” manner.'

In [5]:
art2=''

In [6]:
text=art1+art2

In [7]:
#import datasets
from transformers import BertTokenizer, EncoderDecoderModel

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = EncoderDecoderModel.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail")
model.to("cuda")

batch_size = 8 #16  # change to 64 for full evaluation

# map data correctly
def generate_summary(text):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    # cut off at BERT max length 512
    inputs = tokenizer(text, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    #print(output_str)

    #batch["pred"] = output_str

    return output_str

Downloading: 100%|██████████| 3.66k/3.66k [00:00<00:00, 1.83MB/s]
Downloading: 100%|██████████| 990M/990M [02:07<00:00, 7.74MB/s]
Some weights of EncoderDecoderModel were not initialized from the model checkpoint at patrickvonplaten/bert2bert_cnn_daily_mail and are newly initialized: ['decoder.bert.pooler.dense.weight', 'decoder.bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
generate_summary(text)

['the 11th round of india - china military dialogue may take place on april 9. a score of chinese soldiers still to restore status quo ante in the gogra - hot springs area, or patrolling points 15 and 17 as per the indian military lexicon. the last round of military talks to resolve border tensions last may was held on february 20.']

In [57]:
import pandas as pd
clustered=pd.read_csv('manual_cluster.csv', header=None)
article_list=pd.read_csv('article_list.csv')

In [58]:
import transformers
import pandas as pd 

#import datasets
from transformers import BertTokenizer, EncoderDecoderModel

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = EncoderDecoderModel.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail")
model.to("cuda")

batch_size = 8 #16  # change to 64 for full evaluation

# map data correctly
def generate_summary(articles):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    # cut off at BERT max length 512
    text=' '.join(articles)
    inputs = tokenizer(text, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = model.generate(input_ids, attention_mask=attention_mask)

    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return output_str

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [59]:
summ_titles = []

summ_texts = []for cluster in clustered.groupby(2).groups:
    
article_index = dict(clustered.groupby(2).groups)[cluster]
     article_ids = list(clustered.iloc[article_index,0])
     summ_titles.append(generate_summary(list(article_list[article_list.iloc[:,0].isin(article_ids)]['title'])))  forasumm_text.asppend(generate_summary(list(rticle_list[article_list.iloc[:,0].isin(article_ids)]['text']
)))
 
 
psumm_articles = d.DataFrame(
    
{
        
'summ_titles' : summ_titles,
         'summ_texts' : summ_texts   })  a

In [61]:
summ_articles

,summ_titles,summ_texts
0,[hosps ask attendants to shift patientssmaller...,[deputy commissioner of police atul thakur rec...
1,[madras hcmadras high court takes suo motu not...,[tamil nadu government says it has enough supp...
2,"[ward boy among 3 held for hoarding, selling r...",[two injections and mobile phones were seized ...
3,"[reportsbritain's bans on travel from india, p...",[canada's government will temporarily bar pass...
4,"[thief leaves 1, 710 vaccine doses with an apo...","[thief stole 1, 710 doses of covid - 19 doses ..."
5,[pfizer offers covid - 19 vaccine to indian go...,[us drug major pfizer says it will offer covid...
6,"[hccountry is being run by god, says director ...",[the delhi high court said the reports should ...
7,[cbse introduces changes in assessment and eva...,[the central board of secondary education chan...
8,[west bengal campaign on friday will be the la...,[west bengal chief minister narendra modi canc...
9,[night curfew hits street food vendorsconfusio...,[the government has allowed food delivery from...


In [62]:
summ_articles.to_csv('summ_articles.csv', index=False)

In [55]:
list(article_list[article_list.iloc[:,0].isin(article_ids)]['title'])

['Delhi police on their feet to keep the Capital breathing',
 'Medical O2 crisis increases in Varanasi, hosps ask attendants to shift patients',
 'Smaller hospitals say vendors stretched beyond capacity, some stop admitting patients',
 'Cops restore O2 supply in hospital, save 350 lives',
 'Oxygen supply at Noida hospital chain with 140 Covid patients on last legs',
 'With O2 running out, Delhi-NCR hosps asking patients to leave',
 'Firemen plug O2 pipe leak in NMC hosp, save 42 Covid-19 patients']

In [49]:
article_list.iloc[:,0]

0      176
1       32
2      309
3      107
4      110
5      416
6      114
7      422
8      478
9      204
10     151
11     261
12     259
13     228
14      43
15     319
16     361
17      40
18      19
19      82
20     355
21      73
22     101
23     287
24     215
25     439
26     203
27     111
28     461
29      28
      ... 
509    489
510    128
511     56
512    411
513    199
514    316
515     16
516    105
517     22
518    464
519    421
520    235
521    454
522    430
523      1
524    124
525    137
526    384
527    453
528    389
529    184
530    351
531    191
532    286
533    290
534    288
535    305
536    112
537    321
538    474
Name: Unnamed: 0, Length: 539, dtype: int64